# Jonathan Halverson
# Monday, November 20, 2017
# Transfer learning: Part 2

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('halverson')

In [2]:
tf.reset_default_graph()

In [3]:
#X = tf.placeholder(tf.float32, shape=(None, 10), name='X')
#Z = tf.Variable(tf.random_uniform([100+1, 1], -1, 1), name="Z")
#y = tf.constant(5.6, dtype=tf.float32, name='y')

In [4]:
#init = tf.global_variables_initializer()

In [5]:
#saver = tf.train.Saver()

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
def filter_digits(A, b, i):
     msk = (b <= i)
     return A[msk].copy(), b[msk].copy()

In [8]:
images_train_04, labels_train_04 = filter_digits(mnist.train.images, mnist.train.labels, 4)
images_test_04, labels_test_04 = filter_digits(mnist.test.images, mnist.test.labels, 4)

In [9]:
def fetch_batch(A, b, batch_size):
     # could use randint and bootstrapping
     indices = np.random.choice(range(A.shape[0]), size=batch_size, replace=False)
     return A[indices], b[indices]

Add the previous graph to the default graph:

In [10]:
svr = tf.train.import_meta_graph('/tmp/adam_five_layers.ckpt.meta')

In [11]:
for op in tf.get_default_graph().get_operations():
     print (op.name)

X
y
hidden1/kernel/Initializer/truncated_normal/shape
hidden1/kernel/Initializer/truncated_normal/mean
hidden1/kernel/Initializer/truncated_normal/stddev
hidden1/kernel/Initializer/truncated_normal/TruncatedNormal
hidden1/kernel/Initializer/truncated_normal/mul
hidden1/kernel/Initializer/truncated_normal
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/Const
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden_layers/hidden1/MatMul
hidden_layers/hidden1/BiasAdd
hidden_layers/hidden1/Elu
hidden2/kernel/Initializer/truncated_normal/shape
hidden2/kernel/Initializer/truncated_normal/mean
hidden2/kernel/Initializer/truncated_normal/stddev
hidden2/kernel/Initializer/truncated_normal/TruncatedNormal
hidden2/kernel/Initializer/truncated_normal/mul
hidden2/kernel/Initializer/truncated_normal
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/Const
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
hidden_layers/hidden2/MatMul


In [12]:
#reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='hidden[123]')
#reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
#restore_saver = tf.train.Saver(reuse_vars_dict)
#init = tf.global_variables_initializer()
#saver = tf.train.Saver()

In [ ]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("Adam")

In [ ]:
n_epochs = 10
batch_size = 200

In [ ]:
with tf.Session() as sess:
     svr.restore(sess, '/tmp/adam_five_layers.ckpt')
     for epoch in range(n_epochs):
          for iteration in range(mnist.train.num_examples // batch_size):
               X_batch, y_batch = fetch_batch(images_train_04, labels_train_04, batch_size)
               sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
     accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
     print(epoch, "Test accuracy:", accuracy_val)

In [ ]:
graph = tf.get_default_graph()
     assign_kernel = graph.get_operation_by_name('hidden1/kernel/Assign')
     init_kernel = assign_kernel.inputs[1]
     print init_kernel.eval()
     X_batch = images_train_04[0:3]
     y_batch = labels_train_04[0:3]
     acc = accuracy_op.eval(feed_dict={X:X_batch, y:y_batch})

In [ ]:
with tf.Session() as sess:
     svr.restore(sess, '/tmp/adam_five_layers.ckpt')
     print tf.get_collection('train_op')
     print hidden1.get_shape()
     X_new = images_train_04[0:3]
     Z = logits.eval(feed_dict={X:X_new})
     y_pred = np.argmax(Z, axis=1)